## 关于下载的问题

有关文章[pyhanlp手动配置](https://github.com/hankcs/pyhanlp/wiki/%E6%89%8B%E5%8A%A8%E9%85%8D%E7%BD%AE)

在该部分提到了，在手动配置的时候，如果没有下载data文件，可以手动下载并放置到合适的位置。不过你还可以直接
```python
from pyhanlp import *
```
如果缺少Data 与 java文件，可以在使用（import）时 自动获取与下载

而对于一些缺少其他数据如搜狗实验室的语料时，也自动下载，唯一不能自动下载的是酒店评论数据，这个我建议手动下载，或者使用我在[Github提供的数据](https://github.com/FontTian/pyhanlp_user_guide)。


## 合理的使用HanLP.Config

合理的使用HanLP.Config，而非直接`Config = JClass("com.hankcs.hanlp.HanLP$Config")`获取。

比如设置不显示词性或者字符正则化的时候（`HanLP.Config.Normalization = True`）

In [3]:
# 合理的使用Config文件
# 功能一，设置不显示词性。能够直接获取无词性的输出结果（如：['你好‘，’世界‘]
text = "小区居民有的反对喂养流浪猫，而有的居民却赞成喂养这些小宝贝"
HanLP.Config.ShowTermNature = False
text = "小区居民有的反对喂养流浪猫"
CRFnewSegment = HanLP.newSegment("crf")
# seg分词与segment分词输出结果不一致.此处适合seg
term_list = CRFnewSegment.seg(text)
print(term_list)

[小区, 居民, 有的, 反对, 喂, 养, 流, 浪, 猫]


## 合理的使用SafeJClass

SafeJClass 是hacks在pyhanlp的__init__.py文件中定义的，线程安全的JClass。虽然HanLP很多部分已经实现了线程安全。但是多注意一下总是好的。

In [5]:
# 使用SafeJClass 可以更安全
CoreStopWordDictionary = SafeJClass("com.hankcs.hanlp.dictionary.stopword.CoreStopWordDictionary")
# 配合使用自带停用词功能 -----------> 添加停用词,自定义用户词典等请参阅分词部分
term_list = CRFnewSegment.seg(text)
print(term_list)
CoreStopWordDictionary.apply(term_list)
print(term_list)

[小区, 居民, 有的, 反对, 喂, 养, 流, 浪, 猫]
[小区, 居民, 反对, 养, 流, 浪, 猫]


## 注意开启实体命名识别

之前我们已经演示过了，很多命名实体识别是不默认开启的，所以要注意这个。

In [6]:
"""
HanLP开启命名实体识别

"""

# 音译人名示例
CRFnewSegment = HanLP.newSegment("crf")
term_list = CRFnewSegment.seg("译智社的田丰要说的是这只是一个hanlp命名实体识别的例子")
print(term_list)


print("\n========== 命名实体开启与关闭对比试验 ==========\n")
sentences =[
    "北川景子参演了林诣彬导演的《速度与激情3》",
    "林志玲亮相网友:确定不是波多野结衣？",
    "龟山千广和近藤公园在龟山公园里喝酒赏花",
]
# 通过HanLP 进行全局设置,但是部分分词器本身可能不支持某项功能
# 部分分词器本身对某些命名实体识别效果较好
HanLP.Config.japaneseNameRecognize = False

viterbiNewSegment = HanLP.newSegment("viterbi").enableJapaneseNameRecognize(True)
CRFnewSegment_new = HanLP.newSegment("crf").enableJapaneseNameRecognize(True)
# segSentence
# CRFnewSegment_2.seg2sentence(sentences)
for sentence in sentences:
    print("crf : ",CRFnewSegment.seg(sentence))
    print("crf_new : ",CRFnewSegment_new.seg(sentence))
    print("viterbi : ",viterbiNewSegment.seg(sentence))

[译智社, 的, 田丰, 要, 说, 的, 是, 这, 只, 是, 一个, hanlp命名, 实体, 识别, 的, 例子]

========== 命名实体开启与关闭对比试验 ==========

crf :  [北川, 景子, 参演, 了, 林诣彬, 导演, 的, 《, 速度, 与, 激情, 3, 》]
crf_new :  [北川, 景子, 参演, 了, 林诣彬, 导演, 的, 《, 速度, 与, 激情, 3, 》]
viterbi :  [北川景子, 参演, 了, 林诣彬, 导演, 的, 《, 速度, 与, 激情, 3, 》]
crf :  [林志玲, 亮相, 网友, :, 确定, 不, 是, 波多野, 结衣, ？]
crf_new :  [林志玲, 亮相, 网友, :, 确定, 不, 是, 波多野, 结衣, ？]
viterbi :  [林志玲, 亮相, 网友, :, 确定, 不是, 波多野结衣, ？]
crf :  [龟, 山, 千, 广, 和, 近藤, 公园, 在, 龟山公园, 里, 喝, 酒, 赏, 花]
crf_new :  [龟, 山, 千, 广, 和, 近藤, 公园, 在, 龟山公园, 里, 喝, 酒, 赏, 花]
viterbi :  [龟山千广, 和, 近藤公园, 在, 龟山, 公园, 里, 喝酒, 赏花]


## 尽量使用原生的词向量

尽管HanLP很强大，但是作为Pythobn用户已经建议使用Python原生的词向量。这是更有效率的。


## 模型保存的问题

模型保存不能用pickle保存，或许可以试试原作者给的java保存方式（需要去原作者给的demo去找）但是我尝试了一下，因为需要类型转换，Python不太清楚怎么进行这个，所以暂时还不太明白怎么做。或许可以试试先静态编译一个java文件。因为笔者目前没有这个需求，所以暂时没有进行新的尝试。欢迎你来进行补充。

## 最后的也是最重要的

建议读者尽量把本人写的用户指南，原作者在pyhanlp和hanlp中给的demo都运行一遍


建议Python用户顺如下序本人写的用户指南>原作者给的hanlp中给的demo>原作者给的pyhanlp的demo，
其实看完我的用户指南基本也就不用看原作者给的python版本的demo了，基本都包含了。